In [2]:
import pandas as pd
import json
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import ast

# PJZ Dataset

In [2]:
pjz_df = pd.read_excel('/home/xogns5037/K-CAT/lth/data/PJZ_en_ko.xlsx')
pjz_df.head(10)

,source,label,message_idx,author,time,en,ko,time.1
0,PJ chats,1,0,decoy,0.611111,Hey Its Mads,"안녕, 나 매즈야","""14:40"""
1,PJ chats,1,0,Billy Joe,0.611111,Hey babes,"안녕, 자기야","""14:40"""
2,PJ chats,1,0,Billy Joe,0.611806,Almost done with work. So glad,거의 일 다 끝났어. 정말 기뻐,"""14:41"""
3,PJ chats,1,0,decoy,0.611806,Hey:) that's good right?,안녕 :) 그거 좋은 거지?,"""14:41"""
4,PJ chats,1,0,Billy Joe,0.612500,Yea,응,"""14:42"""
5,PJ chats,1,0,decoy,0.613194,So what's new lol,그럼 뭐 새로운 일 있어? ㅋㅋ,"""14:43"""
6,PJ chats,1,0,Billy Joe,0.613194,Nothing,없어,"""14:43"""
7,PJ chats,1,0,Billy Joe,0.613194,did u touch it in the shower. Lol,샤워할 때 그거 만졌어? ㅋㅋ,"""14:43"""
8,PJ chats,1,0,decoy,0.613889,No had to hurry up lol,"아니, 빨리 해야 했어 ㅋㅋ","""14:44"""
9,PJ chats,1,0,decoy,0.613889,Hold on a min,잠깐만,"""14:44"""


In [3]:
pjz_df = pjz_df[pjz_df['ko'] != 'na']
pjz_df = pjz_df[['source', 'label', 'message_idx', 'author', 'ko']]
pjz_df.rename(columns={'ko': 'text'}, inplace=True)
pjz_df

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
9055,PJ chats,1,621,decoy,응.
9056,PJ chats,1,621,jackjohnsons7,그래서 너 하트포드 어디 거리에서 살아?
9057,PJ chats,1,621,decoy,ㅋㅋㅋ
9058,PJ chats,1,621,decoy,너한테 안 알려줘.


# AI Hub Dataset

In [4]:
def convert_chat_df(source, start, end):
    annotations_subject = []
    message_idx = []
    author = []
    text = []

    for i in range(start, end):
        for j in range(1, 21):
            if j < 10:
                with open(f'./K-CAT/lth/data/AIHub/labeling/{source}/{source.upper()}_{i}_0{j}.json') as f:
                    source_data = json.load(f)
            else:
                with open(f'./K-CAT/lth/data/AIHub/labeling/{source}/{source.upper()}_{i}_{j}.json') as f:
                    source_data = json.load(f)  
            
            if source_data['info'][0]['annotations']['speaker_type'] == '1:1':
                annotations_subject.append(source_data['info'][0]['annotations']['subject'])
                for t in source_data['info'][0]['annotations']['text'].split('\n'):
                    split_idx = t.find(':')
                    n, s = t[:split_idx], t[split_idx+1:]
                    message_idx.append(f'{i}_{j}')
                    author.append(n.strip())
                    text.append(s.strip())
                    
    df = pd.DataFrame({'source': source, 'label': 0, 'message_idx': message_idx, 'author': author, 'text': text})
    
    return df, Counter(annotations_subject)

In [5]:
kakao_df, k_category_cnt = convert_chat_df('kakao', 899, 909)
facebook_df, f_category_cnt = convert_chat_df('facebook', 101, 111)
instagram_df, i_category_cnt = convert_chat_df('instagram', 61, 71)
nateon_df, n_category_cnt = convert_chat_df('nateon', 22, 32)


In [16]:
all_categories = set(k_category_cnt.keys()) | set(f_category_cnt.keys()) | set(i_category_cnt.keys()) | set(n_category_cnt.keys())

category_data = []

for category in sorted(all_categories):
    category_data.append([
        category,
        k_category_cnt.get(category, 0),
        f_category_cnt.get(category, 0),
        i_category_cnt.get(category, 0),
        n_category_cnt.get(category, 0)
    ])

category_df = pd.DataFrame(category_data, columns=['Category', 'K Count', 'F Count', 'I Count', 'N Count'])
category_df

,Category,K Count,F Count,I Count,N Count
0,가족,8,10,10,5
1,건강,8,10,10,5
2,게임,8,10,10,5
3,계절/날씨,8,10,10,5
4,교육,8,10,11,5
5,교통,8,10,10,5
6,군대,8,10,10,5
7,미용,8,10,9,5
8,반려동물,8,10,10,5
9,방송/연예,8,10,10,5


# SBS Dataset

In [6]:
sbs_df = pd.read_excel('/home/xogns5037/K-CAT/lth/data/SBS/SBS.xlsx')

In [7]:
# 대화 내용 합치기
for i in range(len(sbs_df)):
    chat = []
    for j in range(11, 95):
        content = sbs_df.iloc[i, j]
        if pd.notna(content):
            chat.append(content)
        else:
            break
    sbs_df['대화'][i] = chat

sbs_df.drop(sbs_df.columns[12:95], axis=1, inplace=True)

In [80]:
# 필요한 데이터만 남기기
sbs_df.rename(columns={'Unnamed: 0': 'message_idx', '어플': 'source', '제안내용': 'label', '대화': 'text'}, inplace=True)
sbs_df = sbs_df[['source', 'label', 'message_idx', 'text']]

In [81]:
sbs_df.to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS.csv', index=False)

In [82]:
sbs_df = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS.csv')
sbs_df

,source,label,message_idx,text
0,앙챗,음란채팅,chat0001,"['하이, 아침부터 꼴리네요', 'ㅎㅇ', '아침부터 꼴려서, 고추나 만지고 잇어요..."
1,앙챗,음란채팅,chat0002,"['저기요 시골 학교 뒤에서 밤에 여학생 따먹은얘기 라인으로 듣기만 할래요?', '..."
2,앙챗,음란채팅,chat0003,"['ㅎㅇ', 'ㅎㅇ', '네 반가워요', '저도 반가워요', '야한얘기?', '다른..."
3,앙챗,단순채팅,chat0004,"['아조씨랑 아무얘기ㄱㄱ', '안녕', '하이하이, 오늘 학교 안가심?, 말이 없누..."
4,앙챗,성매수,chat0005,"['조건, 보실분 구해요', '저 학생이에요', '괜찮아요, 학교 다녀요?, ?',..."
...,...,...,...,...
1029,동네친구,성매수,chat1030,"['님요. 시간되나여.', '안녕하세요.', '넹. 만남인가여.', '무슨말씀', ..."
1030,동네친구,단순채팅,chat1031,"['좋은하루되세요', '안녕하세요.', '네네']"
1031,동네친구,단순채팅,chat1032,['안녕하세요 처음이용하는거라 괜찮으면 카톡으로 해요. Win221 친추해주세요~ ...
1032,동네친구,성매수,chat1033,"['안녕하세요??', '안녕하세요~', 'ㅈㄱ인가요?', '정확하게 말씀해주세요',..."


In [83]:
sbs_df['text'] = sbs_df['text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
type(sbs_df['text'][0])

list

In [84]:
sbs_df['label'].value_counts()

label
음란채팅    330
단순채팅    313
성매수     265
음란만남     64
단순만남     59
기타        3
Name: count, dtype: int64

In [86]:
# 채팅 내용 개수
sbs_df['text_len'] = sbs_df['text'].apply(lambda x: len(x))
sbs_df['text_len'].describe()

count    1034.000000
mean       14.519342
std        11.692747
min         1.000000
25%         7.000000
50%        12.000000
75%        19.000000
max        84.000000
Name: text_len, dtype: float64

In [87]:
# 대화 내용이 3개 이하 제거
sbs_df = sbs_df[sbs_df['text_len'] > 3]
sbs_df

,source,label,message_idx,text,text_len
0,앙챗,음란채팅,chat0001,"[하이, 아침부터 꼴리네요, ㅎㅇ, 아침부터 꼴려서, 고추나 만지고 잇어요ㅎ, 네;...",6
2,앙챗,음란채팅,chat0003,"[ㅎㅇ, ㅎㅇ, 네 반가워요, 저도 반가워요, 야한얘기?, 다른 얘기.., 어떤?,...",8
3,앙챗,단순채팅,chat0004,"[아조씨랑 아무얘기ㄱㄱ, 안녕, 하이하이, 오늘 학교 안가심?, 말이 없누…학교갔나...",17
4,앙챗,성매수,chat0005,"[조건, 보실분 구해요, 저 학생이에요, 괜찮아요, 학교 다녀요?, ?, 네 학교 ...",21
5,앙챗,성매수,chat0006,"[두번 30에 대나여? 소개부탁드립니다, 저 학생인데여;;, 만남 안하시나영.?, ...",4
...,...,...,...,...,...
1025,동네친구,단순채팅,chat1026,"[안녕?? 어떤 성격이야?, 왜요, 마음통하는 사람 찾는다길래 ㅋ, 아 저 16살인...",33
1026,동네친구,성매수,chat1027,"[마음? 몸?, 안녕하세요 무슨 말씀이신지 ㅎ??, 몸 맞는 사람은 안찾아?, 저 ...",37
1029,동네친구,성매수,chat1030,"[님요. 시간되나여., 안녕하세요., 넹. 만남인가여., 무슨말씀, 조건? 친구?,...",32
1032,동네친구,성매수,chat1033,"[안녕하세요??, 안녕하세요~, ㅈㄱ인가요?, 정확하게 말씀해주세요, 조건인가요?,...",35


In [ ]:
### 채팅 검수를 위한 데이터 분리 ###
sbs_df.drop(['text_len'], axis=1, inplace=True)
sbs_df[sbs_df['label'] == '음란채팅'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_음란채팅.csv', index=False)
sbs_df[sbs_df['label'] == '단순채팅'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_단순채팅.csv', index=False) # 그루밍과 관련 없음 -> 일상 대화
sbs_df[sbs_df['label'] == '성매수'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_성매수.csv', index=False)
sbs_df[sbs_df['label'] == '음란만남'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_음란만남.csv', index=False)
sbs_df[sbs_df['label'] == '단순만남'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_단순만남.csv', index=False) # 그루밍과 관련 없음 -> 일상 대화
sbs_df[sbs_df['label'] == '기타'].to_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_기타.csv', index=False) # 그루밍과 관련 없음


### 검수 완료 후 데이터 합치기 ###
data_1 = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_음란채팅.csv')
data_2 = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_단순채팅.csv')
data_3 = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_성매수.csv')
data_4 = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_음란만남.csv')
data_5 = pd.read_csv('/home/xogns5037/K-CAT/lth/data/SBS/SBS_단순만남.csv')

sbs_grooming_df = pd.concat([data_1, data_2, data_3, data_4], ignore_index=True)
sbs_non_grooming_df = pd.concat

In [73]:
sbs_df['text'][4]

['조건, 보실분 구해요',
 '저 학생이에요',
 '괜찮아요, 학교 다녀요?, ?',
 '네 학교 다녀요',
 '짐, 학교 아니에요?',
 '학교에요',
 '학교, 화장실에서, 자위해본적, 잇어요?',
 '없어요',
 '아다에요?',
 '몰라요ㅎ',
 '섹스, 경험잇네, 어린년이ㅋㅋㅋ, 남친이랑함?',
 '그게 궁금함?ㅎ',
 '응ㅋㅋㅋㅋ, 너 몇살이야',
 16,
 '중딩인데, 섹스 누구랑한거야',
 '안했어',
 '그럼 왜 몰라라함거야, 병신아, ㅋㅋㅋㅋ',
 '그냥;;',
 '니, 보지 똥구멍, 빨고싶다',
 'ㅈㅅ',
 'ㅋㅋㅋ, 씨발련']

# Final Dataset

In [6]:
final_df = pd.concat([pjz_df, kakao_df, facebook_df, instagram_df, nateon_df])
final_df.reset_index(drop=True, inplace=True)
final_df

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
24248,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
24249,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
24250,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
24251,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [7]:
final_df.to_csv('./K-CAT/lth/data/GroomingDataset.csv', index=False)

In [8]:
final_df = pd.read_csv('./K-CAT/lth/data/GroomingDataset.csv')
final_df

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
24248,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
24249,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
24250,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
24251,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.
